In [1]:
import random
import numpy
from operator import add, sub, mul, mod
#TODO assign numbers to variables below
crossoverProbability = 0.6
carryPercentage = 0.2
populationSize = 68


#Function Description

makeFirstPopulation: This function randomly creates an equation(which is a chromosome) by assigning the even bits to operands and odd bits to operators(which are genes)
 
createMatingPool: this function pick some of the chromosomes(according to the carry percentage) and from those recrates the pool by the fitness of each chromosome(their probability is proportional to the fitness)

createCrossoverPool: this function applys crossover to the input

mutate: this function randomly changes each bit; if its an operator it could change it to any other operator and if its operand it changes it by 1 step at each run(for fine tuning)

In [2]:
class EquationBuilder:
    
    def __init__(self, operators, operands, equationLength, goalNumber):
        self.operators = operators
        self.operands = operands
        self.equationLength = equationLength
        self.goalNumber = goalNumber

        # Create the earliest population at the begining
        self.population = self.makeFirstPopulation()
        
    def makeFirstPopulation(self):
        #TODO create random chromosomes to build the early population, and return it
        pop = []
        for chrom in range(populationSize):
            eq = []
            for i in range(self.equationLength):
                if i%2 == 0:
                    eq.append(random.choice(self.operands))
                else:
                    eq.append(random.choice(self.operators))
            pop.append(eq)
        return pop
        

    
    def findEquation(self):
        # Create a new generation of chromosomes, and make it better in every iteration
        while (True):
            random.shuffle(self.population)

            fitnesses = []
            for i in range(populationSize):
                #TODO calculate the fitness of each chromosome
                fit = self.calcFitness(self.population[i])
                #TODO return chromosome if a solution is found, else save the fitness in an array
                if fit == 1:
                    return self.population[i]
                else:
                    fitnesses.append(fit)
                

            #TODO find the best chromosomes based on their fitnesses, and carry them directly to the next generation (optional)
            bestChromosome = [x for _,x in sorted(zip(fitnesses,self.population))]
            carriedChromosomes = []
            for i in range(0, int(populationSize*carryPercentage)):
                carriedChromosomes.append(bestChromosome[i]) 

            # A pool consisting of potential candidates for mating (crossover and mutation)    
            matingPool = self.createMatingPool(populationSize - int(populationSize * carryPercentage),fitnesses)

            # The pool consisting of chromosomes after crossover
            crossoverPool = self.createCrossoverPool(matingPool)

            # Delete the previous population
            self.population.clear()

            # Create the portion of population that is undergone crossover and mutation
            for i in range(populationSize - int(populationSize*carryPercentage)):
                self.population.append(self.mutate(crossoverPool[i]))
                
            # Add the prominent chromosomes directly to next generation
            self.population.extend(carriedChromosomes)
            #print(min(fitnesses))

    
    def createMatingPool(self,size,fitnesses):
        #TODO make a brand new custom pool to accentuate prominent chromosomes (optional)
        #TODO create the matingPool using custom pool created in the last step and return it
        prob = [x / sum(fitnesses) for x in fitnesses]
        indexes = list(range(len(fitnesses)))
        selected_index = numpy.random.choice(indexes, size, p=prob)
        matingPool = [self.population[x] for x in selected_index]
        return matingPool
    
    def createCrossoverPool(self, matingPool):
        crossoverPool = []
        for i in range(len(matingPool)):
            if random.random() > crossoverProbability:
                #TODO don't perform crossover and add the chromosomes to the next generation directly to crossoverPool
                crossoverPool.append(matingPool[i])
            else:
                #TODO find 2 child chromosomes, crossover, and add the result to crossoverPool
                chrom1 = random.choice(matingPool)
                chrom2 = random.choice(matingPool)
                point = random.randrange(self.equationLength)
                res = []
                res[:point] = chrom1[:point]
                res[point:] = chrom2[point:]
                crossoverPool.append(res)
        return crossoverPool
    
    def mutate(self, chromosome):
        #TODO mutate the input chromosome 
        for i in range(self.equationLength):
            if random.random() < 0.1:
                if i%2 == 0:
                    if chromosome[i] + 1 in self.operands and chromosome[i] - 1 in self.operands:
                        if random.random() < 0.5:
                            chromosome[i]  = chromosome[i] + 1
                        else:
                            chromosome[i]  = chromosome[i] - 1

                    elif chromosome[i] + 1 in self.operands and chromosome[i] - 1 not in self.operands:
                            chromosome[i] = chromosome[i] + 1

                    else:
                        chromosome[i] = chromosome[i] -1
                    
                else:
                    chromosome[i] = random.choice(self.operators)
        return chromosome

    def calcFitness(self, chromosome):
        #TODO define the fitness measure here
        eq = [str(x) for x in chromosome]
        fit = abs(self.goalNumber - eval(''.join(eq)))
        return 1/(1 + fit)
        


In [5]:
operands = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
operators = ['+', '-', '*', '%']
equationLength = 23
goalNumber = -123456

equationBuilder = EquationBuilder(operators, operands, equationLength, goalNumber)
equation = equationBuilder.findEquation()
equation = [str(x) for x in equation]
print(''.join(equation))

2-8*8*9-2-8*4*8*6*10*4*2


In [4]:
def find(Input):
    ans = 0
    options = {'+': add, '-': sub, '*': mul, '%' : mod}
    option = add 
    for item in Input:
        if item in options:
            option = options[item]
        else:
            number = float(item)
            ans = option(ans, number)
    return ans

Questions:

1. By very low population the changes will be very slow and the diversity is low, so it may take a lot of time to converge
By very high population the calculations will be a lot and the memory complexity will rise as well, also its possible for repetitive chromosomes to occure

2. if the population grows in each generation, if the algorithm goes on for many generations the population will lean to infinity and the calculation time will rise and consume all of the computaional resouces. But it can lead to a better percision if it converges befor the calculation overload.

3. Crossover should merge some well-performing chromosomes and mutation applies changes to a single chromosome. both will help to converge to the optimal answer, crossover does this at a lower rate by keeping the possible features of a good solution. mutation does this semi-randomly. If mutation is totally random it could remove the features of a good answer and diverge. but by keeping the mutation steps small this problem could be solved(not totally random replacement, but instead minor changes to each bit)
a totally random mutation algorithm may not be efficient if used by itself.

4. Having a good initialization could lead to less steps, for this problem it is possible to start where each operand is the midpoint if the goal is neither low or high, if its very low or very high we could set all the operands to min or max.

5. The main reason may be because of too much randomness in the model: if the fitness is too low and we are so close to the goal, the randomness may start to work against the algorithm and the chromosomes diverge, As stated in part 3, minor changes in mutation may help to fine-tuned moves, which can handle very small changes(as only 1 step in fitness diference) and solve this problem.

6. We could set up a timer in the algorithm , as if the fitness stays on a certain number over some time ,we will assume that it's not possible to reach the goal and exit.